In [1]:
import sys
sys.path.append("/kaggle/input/marigold_model/transformers/default/2/marigold_source")
from Marigold.marigold.marigold_depth_pipeline import MarigoldDepthPipeline

2025-08-08 13:40:37.065398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754660437.235409      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754660437.284058      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import os, gc, cv2, torch, h5py, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

# ===== 模型加载 =====
model_path = "/kaggle/input/marigold_model/transformers/default/2/marigold_model"
pipe = MarigoldDepthPipeline.from_pretrained(model_path).to("cuda")

# ===== 路径与数据加载 =====
mat_path = '/kaggle/input/nyuv2-dataset/nyu_depth_v2_labeled.mat'
save_root = '/kaggle/working'
os.makedirs(save_root, exist_ok=True)
vis_dir = os.path.join(save_root, 'visualizations')
os.makedirs(vis_dir, exist_ok=True)

with h5py.File(mat_path, 'r') as file:
    images = file['images'][:]    # (1449, 3, 640, 480)
    depths = file['depths'][:]    # (1449, 640, 480)

images = np.transpose(images, (3, 2, 1, 0))   # (480, 640, 3, 1449)
depths = np.transpose(depths, (2, 1, 0))      # (480, 640, 1449)

idx_list = np.loadtxt('/kaggle/input/nyu2-test/nyu2_test.txt').astype(int) - 1

# ===== 工具函数 =====
def save_comparison_figure(rgb, gt_depth, pred_depth_aligned, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt_depth > 0) & np.isfinite(gt_depth) & np.isfinite(pred_depth_aligned)
    if np.sum(valid) == 0:
        vmin, vmax = 0, 1
    else:
        all_depths = np.concatenate([gt_depth[valid], pred_depth_aligned[valid]])
        vmin = np.percentile(all_depths, 2)
        vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1); plt.imshow(rgb); plt.title("RGB"); plt.axis("off")
    plt.subplot(1, 3, 2); plt.imshow(gt_depth, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")
    plt.subplot(1, 3, 3); plt.imshow(pred_depth_aligned, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)"); plt.colorbar(fraction=0.046, pad=0.04); plt.axis("off")
    plt.tight_layout(); plt.savefig(save_path); plt.close()

def compute_abs_rel(pred, gt, mask):
    valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    return np.mean(np.abs(pred - gt) / gt) if len(gt) > 0 else np.nan

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid], gt[valid]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    denominator = np.sum(pred ** 2)
    return np.sum(gt * pred) / denominator if denominator > 1e-8 else 1.0

# ===== 主评测循环 =====
results = []
for idx in tqdm(idx_list, desc="Evaluating NYUv2 with Marigold"):
    try:
        rgb = images[:, :, :, idx].astype(np.uint8)      # (480, 640, 3)
        gt_depth = depths[:, :, idx].astype(np.float32)  # (480, 640)
        mask = (gt_depth > 0) & np.isfinite(gt_depth)

        # 推理
        input_image = Image.fromarray(rgb)
        with torch.no_grad():
            result = pipe(
                input_image,
                denoising_steps=4,
                ensemble_size=1,
                processing_res=768,
                match_input_res=True,
                batch_size=0,
                show_progress_bar=False,
            )
        pred_depth = result.depth_np

        # 尺寸对齐
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

        # 缩放匹配 + 指标
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale
        abs_rel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1 = compute_delta1(pred_aligned, gt_depth, mask)

        print(f"[{idx}] AbsRel: {abs_rel:.4f}, δ1: {delta1*100:.2f}%")
        results.append([idx, abs_rel, delta1])

        # 可视化
        save_path = os.path.join(vis_dir, f"{idx:04d}_compare.png")
        save_comparison_figure(rgb, gt_depth, pred_aligned, mask, save_path)

        del rgb, gt_depth, mask, pred_depth, pred_aligned, input_image
        torch.cuda.empty_cache(); gc.collect()

    except Exception as e:
        print(f"跳过 {idx}，错误: {e}")
        continue

# ===== 保存整体结果 =====
df = pd.DataFrame(results, columns=['idx', 'AbsRel', 'Delta1'])
df.to_csv(os.path.join(save_root, "marigold_nyuv2_results.csv"), index=False)

# 打包可视化
zip_path = shutil.make_archive(os.path.join(save_root, "visualizations_nyuv2"), 'zip', root_dir=vis_dir)
print(f"可视化图像已打包: {zip_path}")


The config attributes {'prediction_type': 'depth'} were passed to MarigoldDepthPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'prediction_type': 'depth'} are not expected by MarigoldDepthPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Evaluating NYUv2 with Marigold:   0%|          | 0/666 [00:00<?, ?it/s]

[1] AbsRel: 0.2859, δ1: 56.17%


Evaluating NYUv2 with Marigold:   0%|          | 1/666 [00:03<38:54,  3.51s/it]

...

[1444] AbsRel: 0.3124, δ1: 48.11%


Evaluating NYUv2 with Marigold: 100%|█████████▉| 665/666 [43:29<00:05,  5.30s/it]

[1447] AbsRel: 0.3301, δ1: 48.46%


Evaluating NYUv2 with Marigold: 100%|██████████| 666/666 [43:35<00:00,  3.93s/it]


可视化图像已打包: /kaggle/working/visualizations_nyuv2.zip
